# Importing modules

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np
import string

dir_path = os.getcwd()

# Scraping urls and video_ids

In [ ]:
# Retrieve all partij urls
videos = ["/watch?v=0vFTfoOavRM&list=UUt7fwAhXDy3oNFTAzF2o8Pw&index=1"]
index = ["ex=1"]
title = ["Perfume Genius - Set My Heart on Fire Immediately ALBUM REVIEW"]

while len(videos) < 3238:
    response = requests.get(base_url + videos[-1])
    soup = BeautifulSoup(response.text, "html.parser")
    next_video = soup.find_all('a',  {"class": "spf-link playlist-video clearfix yt-uix-sessionlink spf-link"})
    for nv in next_video:
        if nv["href"][-4:] not in index:
            videos.append(nv["href"])
            index.append(nv["href"][-4:])
            title.append(nv.find("h4").text.split("\n")[1].replace("  ", ""))
    
            
    print(len(videos), end="\r")

In [ ]:
df_url = pd.DataFrame(columns = ["url", "id", "title"])
df_url.url = videos
df_url.id = df_url.url.str.split("&").apply(lambda x: x[0].replace("/watch?v=", ""))
df_url.title = title
df_url.to_csv("urls.csv", index = None, sep = ";")

In [99]:
df_url = pd.read_csv("urls.csv", sep = ";")
df_url.title = df_url.title.str.lower()

filter1 = df_url.title.str.contains("review")
filter2 = df_url.title.str.contains("-")
df_title = df_url.loc[(filter1 & filter2), ["id", "title"]]
df_title.title = df_title.title.str.replace("album", "")
df_title.title = df_title.title.str.replace("review", "")

filter3 = df_url.title.str.contains("not good")
df_notgood = df_url.loc[filter3, ["id", "title"]]
df_notgood.title = df_notgood.title.str.replace("not good", "")
df_notgood.title = df_notgood.title.str.replace(": ", "")

In [76]:
df_artist_album = pd.DataFrame(columns = ["id", "artist", "album"])
df_artist_album.id = df_title.id
df_artist_album.artist = df_title.title.str.split("-").apply(lambda x: x[0]).str.strip()
df_artist_album.album = df_title.title.str.split("-").apply(lambda x: x[-1]).str.strip()

In [110]:
df_notgood_artist_album =  pd.DataFrame(columns = ["id", "artist", "album"])
df_notgood_artist_album.id = df_notgood.id
df_notgood_artist_album.artist = df_notgood.title.str.split("'s").apply(lambda x: x[0]).str.strip()
df_notgood_artist_album.album = df_notgood.title.str.split("'s").apply(lambda x: x[-1]).str.strip()
df_artist_album = df_artist_album.append(df_notgood_artist_album).sort_index()

In [111]:
df_artist_album.to_csv("albums.csv", index = False, sep = ";")